In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutomaticSpeechRecognitionPipeline

from scipy.special import softmax
import numpy as np
import pandas as pd

df = pd.read_csv('pokemon_ultrasunmoon.csv')

In [25]:
len(df)

47588

In [26]:
df = df[df['text'].str.contains('I liked a @YouTube video')== False]
df = df[df['text'].str.contains('I added a video to a @YouTube')== False]

In [27]:
len(df)

33461

In [28]:
# Preprocessing
preprocessed_text = []

for tweet in df['text']:
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = 'http'
        tweet_words.appendd(word)
    tweet_proc = " ".join(tweet_words)
    preprocessed_text.append(tweet_proc)

In [30]:
df['preprocessed tweets'] = preprocessed_text
# df.drop(columns=df.columns[0], axis=1, inplace=True)

In [31]:
df.reset_index(drop=True, inplace=True)
long_tweets = df['preprocessed tweets'].str.len().sort_values(ascending=False)

long_tweets

6323     303
10240    301
2700     295
8811     294
1453     294
        ... 
15159     12
25996     11
4668      11
27760     10
32280      9
Name: preprocessed tweets, Length: 33461, dtype: int64

In [32]:
long_tweets.head(10)

6323     303
10240    301
2700     295
8811     294
1453     294
9219     293
6210     289
5518     288
2779     288
6537     287
Name: preprocessed tweets, dtype: int64

In [19]:
# df.at[300415,'preprocessed tweets'] = 'I\'m ready for you, my "son" #PokemonSunMoon http'
# df.at[58395,'preprocessed tweets'] = 'Litten #pokemonsunmoon'
# df.at[99361,'preprocessed tweets'] = ' '
# df.at[287842,'preprocessed tweets'] ='Nid sum cash 2 buyyyy pokemon moon this fridayyyy,haaaalllpppp'
# df.at[66717,'preprocessed tweets'] = 'New Pokemon Sun and Moon theory [troll] coming tomorrow'
# df.at[295865,'preprocessed tweets']= 'Alright gays in less than 2Ô hours Pokemon Sun and Pokemon Moon will officially'
# df.at[67278,'preprocessed tweets'] = "ALSO I just wanted to express how absolutely excited I am for Pokemon sun&amp;moon LIKE I'M SO"

In [34]:
# Load model and tokenizer

roberta = "cardiffnlp/twitter-roberta-base-sentiment-latest"
#roberta = "cardiffnlp/twitter-roberta-base-2021-124m"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
sentiment_scores = []

for processed_tweet in df['preprocessed tweets']:
    encoded_tweet = tokenizer(processed_tweet, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    sentiment_scores.append(scores)

In [36]:
df['sentiment scores'] = sentiment_scores

In [37]:
labels = ['Negative', 'Neutral', 'Positive']
index_no = [0,1,2]

label_dict = dict(zip(index_no, labels))

overall_sentiment = []

for sentiment_score in df['sentiment scores']:
    overall_sentiment.append(label_dict.get(sentiment_score.argmax()))

In [38]:
df['sentiment'] = overall_sentiment

In [ ]:
from datetime import datetime

df['created at'] = df['created at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))


In [40]:
df.to_csv('pokemon_ultrasm_tweets_datatset.csv')